In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install faiss-cpu --no-cache
!pip install autofaiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 49 kB/s 
     |████████████████████████████████| 199 kB 63.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=907c1182800e451771c09bc3e25e7f614c9cf26c120ce7c66d194987b48f73d0
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 13.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 5.3 MB/s 
     |████████████████████████████████| 87 kB 5.1 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=36463371263b8d6fadcda481fa91b4bbe9394

In [ ]:
import numpy as np
import pandas as pd
import sys
import time
import os
import shutil
import faiss
import glob
import numpy as np
from pyspark.sql import SparkSession

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
from operator import add
from sklearn.neighbors import KNeighborsClassifier

dataset


In [ ]:
from sklearn.datasets import fetch_covtype
forest = fetch_covtype()
Data= forest['data']
label = forest['target']
data_cov = np.concatenate([Data[:, :32], label.reshape(-1, 1)], axis = -1)

In [ ]:
data_cov = data_cov[:10000, :]
data_cov.shape

(10000, 33)

In [ ]:
all_cov, query_cov = train_test_split(data_cov, test_size=0.2, random_state=4)

faiss

In [ ]:
os.makedirs("my_index_folder", exist_ok=True)
max_index_query_time_ms = 10 #@param {type: "number"}
max_index_memory_usage = "10MB" #@param
metric_type = "l2" #@param ['ip', 'l2']

build index

In [ ]:
def build_index(all_data_np, query_data_np, dim, param, measure = faiss.METRIC_L2):

    total_num = len(all_data_np)
    total_index = []
    for pi in range(10):
        if pi < 9:
            st, end = total_num // 10 * pi,  total_num // 10 * (pi + 1)
        else:
            st, end = total_num // 10 * pi,  total_num
        index = faiss.index_factory(dim, param, measure)

        xb = np.float32(all_data_np[st:end, :-1])
        index.train(xb)
        index.add(xb)
        total_index.append(index)
        
    return total_index

In [ ]:
def build_auto_index(all_data_np):
    for pi in range(10):
        embeddings_dir = "tmp_folder" + str(pi)
        if os.path.exists(embeddings_dir):
          shutil.rmtree(embeddings_dir)
        os.makedirs(embeddings_dir)
        knn_index = "knn" + str(pi) + ".index"
        infos_path = "infos" + str(pi) + ".json"

        total_num = len(all_data_np)
        if pi < 9:
            st, end = total_num // 10 * pi,  total_num // 10 * (pi + 1)
        else:
            st, end = total_num // 10 * pi,  total_num

        print('all_data_np[st:end, :-1] = ', all_data_np[st:end, :-1].shape)
        np.save(f"{embeddings_dir}/emb.npy", np.float32(all_data_np[st:end, :-1])) 
        !autofaiss build_index --embeddings={embeddings_dir} \
                            --index_path={knn_index} \
                            --index_infos_path={"infos.json"} \
                            --metric_type={metric_type} \
                            --max_index_query_time_ms=10 \
                            --max_index_memory_usage={max_index_memory_usage}
def build_auto_index2():
    total_index = []
    for i in range(10):
        total_index.append(faiss.read_index("knn" + str(i) + ".index"))
    return total_index

In [ ]:
def PQ_operation(all_data_np, query_data_np, K):
    
    def Distance_FAISS(test, train):
        '''
        Input:
            - test: Numpy Array with label as last value
            - train: Numpy Array with label as last value
        
        Returns distance between test and train arrays
        '''
        st = len(all_data_np) // 10* train[0]
        distances, indices = total_index[train[0]].search(test[:-1].reshape(1, -1), K)
        tmp = []
        for i, (dist, indice) in enumerate(zip(distances[0], indices[0])):
            tmp.append((dist, st + indice))
        return tmp

    spark = SparkSession\
            .builder\
            .appName("PythonKNN")\
            .getOrCreate()

    sc = spark.sparkContext
    sc.setLogLevel("ERROR")

    all_data = np.arange(10)
    df = pd.DataFrame(all_data)
    all_data = spark.createDataFrame(df)

    df = pd.DataFrame(query_data_np)
    query_data = spark.createDataFrame(df).rdd
    query_data = query_data.map(lambda x: np.float32(x))

    start_time = time.time()

    accuracy = 0
    count = 0

    for test_point in query_data.collect():

        distances = all_data.rdd.map(
            lambda train_point: Distance_FAISS(test_point, train_point))
        
        distances = sc.parallelize(distances.reduce(lambda a, b: a + b))
        k_nearest_neighbours = sc.parallelize(distances.takeOrdered(K, key = lambda p: p[0])).map(lambda x: (all_data_np[x[1],-1], 1))
        
        k_nearest_predictions = k_nearest_neighbours.reduceByKey(
            lambda x1, x2: x1 + x2)
        
        predict_label = k_nearest_predictions.takeOrdered(1,
            key = lambda x: -x[1])[0][0]

        if predict_label == test_point[-1]:
            accuracy += 1

        count += 1

    end_time = time.time()

    print('accuracy = ', accuracy / count)
    time_taken = end_time - start_time
    print('time_taken = ', time_taken)

In [ ]:
all_cov.shape

(8000, 33)

In [ ]:
build_auto_index(all_cov)

all_data_np[st:end, :-1] =  (800, 32)
2022-11-15 19:32:48,549 [INFO]: Using 2 omp threads (processes), consider increasing --nb_cores if you have more
2022-11-15 19:32:48,549 [INFO]: Launching the whole pipeline 11/15/2022, 19:32:48
2022-11-15 19:32:48,549 [INFO]: Reading total number of vectors and dimension 11/15/2022, 19:32:48
100% 1/1 [00:00<00:00, 24672.38it/s]
2022-11-15 19:32:48,678 [INFO]: There are 800 embeddings of dim 32
2022-11-15 19:32:48,678 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.1291 secs
2022-11-15 19:32:48,678 [INFO]: 	Compute estimated construction time of the index 11/15/2022, 19:32:48
2022-11-15 19:32:48,678 [INFO]: 		-> Train: 16.7 minutes
2022-11-15 19:32:48,678 [INFO]: 		-> Add: 0.0 seconds
2022-11-15 19:32:48,678 [INFO]: 		Total: 16.7 minutes
2022-11-15 19:32:48,678 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0002 secs
2022-11-15 19:32:48,678 [INFO]: 	Checking that your have enough memory avail

In [ ]:
dim, measure, param = all_cov.shape[1]-1, faiss.METRIC_L2, 'PQ4'
total_index = build_index(all_cov, query_cov, dim, param, measure = faiss.METRIC_L2)
for k in [8, 16,32,64]:
    PQ_operation(all_cov, query_cov, k // 2)

accuracy =  0.772
time_taken =  664.60009932518
accuracy =  0.758
time_taken =  622.645307302475
accuracy =  0.7275
time_taken =  611.6613938808441
accuracy =  0.692
time_taken =  612.2968657016754
